In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
from pathlib import Path
from pprint import pprint
from os import environ

from tqdm import tqdm

import polars as pl

from eott_dataset import *
from eott_dataset.etl import *

In [ ]:
# %env set EOTT_DATASET_PATH
# %env set EOTT_OUTPUT_PATH

output_dir = Path(environ["EOTT_OUTPUT_PATH"]).expanduser().resolve()
print("output directory:")
print(output_dir)

pdf = participant_dataframe().collect()
pdf.write_parquet(output_dir / "participant.parquet", compression="uncompressed")
pdf

In [ ]:
with pl.StringCache():
    dfs = [event_dataframes(p, pid_from_name(p.parent.name)) for p in  glob_log_files()]
    dfs: list[pl.DataFrame]
    for dfs, name in zip(zip(*dfs), ("log", "scroll", "mouse")):
        pl.concat(dfs).write_parquet(output_dir / f"{name}.parquet", compression="lz4")

In [ ]:
df = pl.concat([calibration_dataframe(p, pid_from_name(p.parent.name)) for p in glob_specs_files()])
df.collect().write_parquet(output_dir / "calibration.parquet", compression="uncompressed")

In [ ]:
df = pl.concat([tobii_dataframe(p, pid_from_name(p.parent.name)) for p in glob_tobii_files()])
df.collect().write_parquet(output_dir / "tobii.parquet", compression="lz4")

In [ ]:
# test files are readable
for path in output_dir.glob("*.parquet"):
    print(path)
    df = pl.read_parquet(path, use_statistics=False)
    pprint(df.schema)
    print()
    del df

In [ ]:
df = pl.scan_parquet(output_dir / "webcam.parquet")

In [ ]:
from io import BytesIO
from decord import VideoReader

entry = df.filter(pid=1, index=2, aux=0).collect().to_dicts()[0]
vr = VideoReader(BytesIO(entry["video"]))

for frame in vr:
    print(frame.shape)
del vr

In [ ]:
raise
with tqdm(desc="webcam", total=len([*glob_webcam_files()])) as t:
    df = webcam_dataframe(callback=lambda: t.update(1))
    df.sink_parquet(output_dir / "webcam.parquet", compression="uncompressed")